In [1]:
# Import dependencies
import pandas as pd
import numpy as np
%matplotlib inline

from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

In [18]:
# Read csv file
data = pd.read_csv('./Resources/Sales_Area_Data/Sales/final_sales_data.csv')

# Keep only rows from 2019 for training; drop 2020 and 2021 data
index_names = data[data['year'] != 2019].index
data.drop(index_names, inplace = True)

data = data.sort_values(by=['zipcode', 'month'], axis=0)
  
data.head()

,Unnamed: 0,zipcode,year,month,total_sales,avg_sale_price,med_sale_price,mortgage_rate
5589,5589,32003,2019,1,32,289290.0,244950.0,4.4640
5700,5700,32003,2019,2,41,286173.0,270000.0,4.3700
5804,5804,32003,2019,3,21,301620.0,275000.0,4.2650
5908,5908,32003,2019,4,42,263475.0,264500.0,4.1425
6022,6022,32003,2019,5,66,293816.0,281000.0,4.0720


In [19]:
data = data.groupby(['zipcode','month'])['avg_sale_price'].agg(['sum']).reset_index()
data = data.pivot_table(values='sum', columns='month', index='zipcode')
data.columns.name = None
data = data.reset_index()

data.head()

,zipcode,1,2,3,4,5,6,7,8,9,10,11,12
0,32003,289290.0,286173.0,301620.0,263475.0,293816.0,298865.0,295053.0,280793.0,299109.0,293246.0,307094.0,312718.0
1,32008,94875.0,106250.0,NaN,132475.0,76825.0,153000.0,131322.0,168143.0,126875.0,104500.0,139929.0,178917.0
2,32009,275000.0,99000.0,112000.0,NaN,113994.0,150220.0,227400.0,NaN,150333.0,201667.0,177115.0,352500.0
3,32011,219310.0,152160.0,160508.0,171022.0,210804.0,144637.0,220224.0,170269.0,214582.0,193573.0,146888.0,150112.0
4,32024,252829.0,114341.0,87738.0,157358.0,147920.0,137455.0,157163.0,108509.0,145418.0,170477.0,192769.0,150936.0


In [20]:
melt = data.melt(id_vars='zipcode', var_name='month', value_name='avg_sale_price')

melt['zipcode'] = melt['zipcode']
melt['month'] = melt['month']

melt = melt.sort_values(['month', 'zipcode'])
melt.head(3000)

,zipcode,month,avg_sale_price
0,32003,1,289290.0
1,32008,1,94875.0
2,32009,1,275000.0
3,32011,1,219310.0
4,32024,1,252829.0
...,...,...,...
2995,32343,4,136365.0
2996,32344,4,146219.0
2997,32346,4,214350.0
2998,32347,4,87338.0


In [21]:
index = melt.index
number_of_rows = len(index)
 
print(number_of_rows)

11412


In [22]:
melt2 = melt.copy()
melt2['last_month_sales'] = melt2.groupby(['zipcode'])['avg_sale_price'].shift()
melt2['last_month_diff'] = melt2.groupby(['zipcode'])['last_month_sales'].diff()
melt2 = melt2.dropna()
melt2.head()

,zipcode,month,avg_sale_price,last_month_sales,last_month_diff
1902,32003,3,301620.0,286173.0,-3117.0
1904,32009,3,112000.0,99000.0,-176000.0
1905,32011,3,160508.0,152160.0,-67150.0
1906,32024,3,87738.0,114341.0,-138488.0
1907,32025,3,100786.0,118032.0,16522.0


In [77]:
index = melt2.index
number_of_rows = len(index)
 
print(number_of_rows)

8762


In [34]:
def rmsle(ytrue, ypred):
    return np.sqrt(mean_squared_log_error(ytrue, ypred))

### Establishing Baseline

In [79]:
mean_error = []
for month in range(11,13):
    train = melt2[melt2['month'] < month]
    val = melt2[melt2['month'] == month]

    p = val['last_month_sales'].values

    error = rmsle(val['avg_sale_price'].values, p)
    print('Month %d - Error %.5f' % (month, error))
    mean_error.append(error)

print('Mean Error = %.5f' % np.mean(mean_error))

Month 11 - Error 0.26051
Month 12 - Error 0.28529
Mean Error = 0.27290


### Training

In [80]:
mean_error = []
for month in range(11,13):
    train = melt2[melt2['month'] < month]
    val = melt2[melt2['month'] == month]

    xtr, xts = train.drop(['avg_sale_price'], axis=1), val.drop(['avg_sale_price'], axis=1)
    ytr, yts = train['avg_sale_price'].values, val['avg_sale_price'].values

    model = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)
    model.fit(xtr, ytr)

    p = model.predict(xts)

    error = rmsle(yts, p)
    print('Month %d - Error %.5f' % (month, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))

Month 11 - Error 0.24223
Month 12 - Error 0.24292
Mean Error = 0.24258


In [81]:
melt3 = melt.copy()
melt3['Last_Month_Sales'] = melt3.groupby(['zipcode'])['avg_sale_price'].shift()
melt3['Last_Month_Diff'] = melt3.groupby(['zipcode'])['Last_Month_Sales'].diff()
melt3['Last-1_Month_Sales'] = melt3.groupby(['zipcode'])['avg_sale_price'].shift(2)
melt3['Last-1_Month_Diff'] = melt3.groupby(['zipcode'])['Last-1_Month_Sales'].diff()
melt3 = melt3.dropna()
melt3.head()

,zipcode,month,avg_sale_price,Last_Month_Sales,Last_Month_Diff,Last-1_Month_Sales,Last-1_Month_Diff
2853,32003,4,263475.0,301620.0,15447.0,286173.0,-3117.0
2856,32011,4,171022.0,160508.0,8348.0,152160.0,-67150.0
2857,32024,4,157358.0,87738.0,-26603.0,114341.0,-138488.0
2858,32025,4,110079.0,100786.0,-17246.0,118032.0,16522.0
2859,32033,4,121500.0,243000.0,-5794.0,248794.0,69419.0


In [82]:
mean_error = []
for month in range(11,13):
    train = melt3[melt3['month'] < month]
    val = melt3[melt3['month'] == month]

    xtr, xts = train.drop(['avg_sale_price'], axis=1), val.drop(['avg_sale_price'], axis=1)
    ytr, yts = train['avg_sale_price'].values, val['avg_sale_price'].values

    model = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)
    model.fit(xtr, ytr)

    p = model.predict(xts)

    error = rmsle(yts, p)
    print('Month %d - Error %.5f' % (month, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))

Month 11 - Error 0.23398
Month 12 - Error 0.21673
Mean Error = 0.22536


In [83]:
melt4 = melt.copy()
melt4['Last_Month_Sales'] = melt4.groupby(['zipcode'])['avg_sale_price'].shift()
melt4['Last_Month_Diff'] = melt4.groupby(['zipcode'])['Last_Month_Sales'].diff()
melt4['Last-1_Month_Sales'] = melt4.groupby(['zipcode'])['avg_sale_price'].shift(2)
melt4['Last-1_Month_Diff'] = melt4.groupby(['zipcode'])['Last-1_Month_Sales'].diff()
melt4['Last-2_Month_Sales'] = melt4.groupby(['zipcode'])['avg_sale_price'].shift(3)
melt4['Last-2_Month_Diff'] = melt4.groupby(['zipcode'])['Last-2_Month_Sales'].diff()
melt4 = melt4.dropna()
melt4.head()

,zipcode,month,avg_sale_price,Last_Month_Sales,Last_Month_Diff,Last-1_Month_Sales,Last-1_Month_Diff,Last-2_Month_Sales,Last-2_Month_Diff
3804,32003,5,293816.0,263475.0,-38145.0,301620.0,15447.0,286173.0,-3117.0
3807,32011,5,210804.0,171022.0,10514.0,160508.0,8348.0,152160.0,-67150.0
3808,32024,5,147920.0,157358.0,69620.0,87738.0,-26603.0,114341.0,-138488.0
3809,32025,5,156524.0,110079.0,9293.0,100786.0,-17246.0,118032.0,16522.0
3810,32033,5,156700.0,121500.0,-121500.0,243000.0,-5794.0,248794.0,69419.0


In [84]:
mean_error = []
for month in range(11,13):
    train = melt4[melt4['month'] < month]
    val = melt4[melt4['month'] == month]

    xtr, xts = train.drop(['avg_sale_price'], axis=1), val.drop(['avg_sale_price'], axis=1)
    ytr, yts = train['avg_sale_price'].values, val['avg_sale_price'].values

    model = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)
    model.fit(xtr, ytr)

    p = model.predict(xts)

    error = rmsle(yts, p)
    print('Month %d - Error %.5f' % (month, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))

Month 11 - Error 0.21861
Month 12 - Error 0.21076
Mean Error = 0.21468


In [87]:
melt5 = melt.copy()
melt5['Last_Month_Sales'] = melt5.groupby(['zipcode'])['avg_sale_price'].shift()
melt5['Last_Month_Diff'] = melt5.groupby(['zipcode'])['Last_Month_Sales'].diff()
melt5['Last-1_Month_Sales'] = melt5.groupby(['zipcode'])['avg_sale_price'].shift(2)
melt5['Last-1_Month_Diff'] = melt5.groupby(['zipcode'])['Last-1_Month_Sales'].diff()
melt5['Last-2_Month_Sales'] = melt5.groupby(['zipcode'])['avg_sale_price'].shift(3)
melt5['Last-2_Month_Diff'] = melt5.groupby(['zipcode'])['Last-2_Month_Sales'].diff()
melt5['Last-3_Month_Sales'] = melt5.groupby(['zipcode'])['avg_sale_price'].shift(4)
melt5['Last-3_Month_Diff'] = melt5.groupby(['zipcode'])['Last-3_Month_Sales'].diff()
melt5 = melt5.dropna()
melt5.head()

,zipcode,month,avg_sale_price,Last_Month_Sales,Last_Month_Diff,Last-1_Month_Sales,Last-1_Month_Diff,Last-2_Month_Sales,Last-2_Month_Diff,Last-3_Month_Sales,Last-3_Month_Diff
4755,32003,6,298865.0,293816.0,30341.0,263475.0,-38145.0,301620.0,15447.0,286173.0,-3117.0
4758,32011,6,144637.0,210804.0,39782.0,171022.0,10514.0,160508.0,8348.0,152160.0,-67150.0
4759,32024,6,137455.0,147920.0,-9438.0,157358.0,69620.0,87738.0,-26603.0,114341.0,-138488.0
4760,32025,6,139078.0,156524.0,46445.0,110079.0,9293.0,100786.0,-17246.0,118032.0,16522.0
4761,32033,6,180849.0,156700.0,35200.0,121500.0,-121500.0,243000.0,-5794.0,248794.0,69419.0


In [88]:
mean_error = []
for month in range(11,13):
    train = melt5[melt5['month'] < month]
    val = melt5[melt5['month'] == month]

    xtr, xts = train.drop(['avg_sale_price'], axis=1), val.drop(['avg_sale_price'], axis=1)
    ytr, yts = train['avg_sale_price'].values, val['avg_sale_price'].values

    model = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)
    model.fit(xtr, ytr)

    p = model.predict(xts)

    error = rmsle(yts, p)
    print('Month %d - Error %.5f' % (month, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))

Month 11 - Error 0.21694
Month 12 - Error 0.20501
Mean Error = 0.21097


In [89]:
melt6 = melt.copy()
melt6['Last_Month_Sales'] = melt6.groupby(['zipcode'])['avg_sale_price'].shift()
melt6['Last_Month_Diff'] = melt6.groupby(['zipcode'])['Last_Month_Sales'].diff()
melt6['Last-1_Month_Sales'] = melt6.groupby(['zipcode'])['avg_sale_price'].shift(2)
melt6['Last-1_Month_Diff'] = melt6.groupby(['zipcode'])['Last-1_Month_Sales'].diff()
melt6['Last-2_Month_Sales'] = melt6.groupby(['zipcode'])['avg_sale_price'].shift(3)
melt6['Last-2_Month_Diff'] = melt6.groupby(['zipcode'])['Last-2_Month_Sales'].diff()
melt6['Last-3_Month_Sales'] = melt6.groupby(['zipcode'])['avg_sale_price'].shift(4)
melt6['Last-3_Month_Diff'] = melt6.groupby(['zipcode'])['Last-3_Month_Sales'].diff()
melt6['Last-4_Month_Sales'] = melt6.groupby(['zipcode'])['avg_sale_price'].shift(5)
melt6['Last-4_Month_Diff'] = melt6.groupby(['zipcode'])['Last-4_Month_Sales'].diff()

melt6 = melt6.dropna()
melt6.head()

,zipcode,month,avg_sale_price,Last_Month_Sales,Last_Month_Diff,Last-1_Month_Sales,Last-1_Month_Diff,Last-2_Month_Sales,Last-2_Month_Diff,Last-3_Month_Sales,Last-3_Month_Diff,Last-4_Month_Sales,Last-4_Month_Diff
5706,32003,7,295053.0,298865.0,5049.0,293816.0,30341.0,263475.0,-38145.0,301620.0,15447.0,286173.0,-3117.0
5709,32011,7,220224.0,144637.0,-66167.0,210804.0,39782.0,171022.0,10514.0,160508.0,8348.0,152160.0,-67150.0
5710,32024,7,157163.0,137455.0,-10465.0,147920.0,-9438.0,157358.0,69620.0,87738.0,-26603.0,114341.0,-138488.0
5711,32025,7,128616.0,139078.0,-17446.0,156524.0,46445.0,110079.0,9293.0,100786.0,-17246.0,118032.0,16522.0
5712,32033,7,241143.0,180849.0,24149.0,156700.0,35200.0,121500.0,-121500.0,243000.0,-5794.0,248794.0,69419.0


In [90]:
mean_error = []
for month in range(11,13):
    train = melt6[melt6['month'] < month]
    val = melt6[melt6['month'] == month]

    xtr, xts = train.drop(['avg_sale_price'], axis=1), val.drop(['avg_sale_price'], axis=1)
    ytr, yts = train['avg_sale_price'].values, val['avg_sale_price'].values

    model = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)
    model.fit(xtr, ytr)

    p = model.predict(xts)

    error = rmsle(yts, p)
    print('Month %d - Error %.5f' % (month, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))

Month 11 - Error 0.21520
Month 12 - Error 0.20217
Mean Error = 0.20869


### Changing the fit

In [91]:
mean_error = []
for week in range(11,13):
    train = melt6[melt6['month'] < month]
    val = melt6[melt6['month'] == month]

    xtr, xts = train.drop(['avg_sale_price'], axis=1), val.drop(['avg_sale_price'], axis=1)
    ytr, yts = train['avg_sale_price'].values, val['avg_sale_price'].values

    mdl = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)
    mdl.fit(xtr, np.log1p(ytr))

    p = np.expm1(mdl.predict(xts))

    error = rmsle(yts, p)
    print('Month %d - Error %.5f' % (week, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))

Month 11 - Error 0.20384
Month 12 - Error 0.20384
Mean Error = 0.20384
